In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, f1_score, accuracy_score, roc_auc_score

In [2]:
df=pd.read_csv('/content/train.csv', delimiter=',')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Удалим признаки, которые могут привести к переобучению.

In [3]:
df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


Заполним отсутствующие значения в столбце "Age" медианным значением. Удалим строки, в котрых отсутствуют значения.

In [4]:
df['Age']=df['Age'].fillna(df['Age'].median())
df=df.dropna()
df=df.reset_index(drop=True)

Перобразуем категориальные признаки.

In [5]:
ohe = OneHotEncoder(sparse=False,drop='first')
df_ohe = ohe.fit_transform(df[['Sex','Embarked']])
df_ohe=pd.DataFrame(df_ohe)
df_ohe.columns=['Sex','Emb_Q','Emb_S']
df_ohe.head()

,Sex,Emb_Q,Emb_S
0,1.0,0.0,1.0
1,0.0,0.0,0.0
2,0.0,0.0,1.0
3,0.0,0.0,1.0
4,1.0,0.0,1.0


Удалим из датафрейма столбцы с категориальными значениями.

In [6]:
df_drop=df.drop(columns=['Sex', 'Embarked'])
df_drop.head()

,Survived,Pclass,Age,SibSp,Parch,Fare
0,0,3,22.0,1,0,7.2500
1,1,1,38.0,1,0,71.2833
2,1,3,26.0,0,0,7.9250
3,1,1,35.0,1,0,53.1000
4,0,3,35.0,0,0,8.0500


Добавим к датафрейму преобразованные категориальные данные.

In [7]:
df_result= pd.concat([df_drop, df_ohe], sort=False,axis=1)
df_result.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex,Emb_Q,Emb_S
0,0,3,22.0,1,0,7.2500,1.0,0.0,1.0
1,1,1,38.0,1,0,71.2833,0.0,0.0,0.0
2,1,3,26.0,0,0,7.9250,0.0,0.0,1.0
3,1,1,35.0,1,0,53.1000,0.0,0.0,1.0
4,0,3,35.0,0,0,8.0500,1.0,0.0,1.0


Делим датафрейм на обучающий и тестовый фрагменты.

In [8]:
train, test = train_test_split(df_result,test_size=0.3)

In [9]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 267 entries, 150 to 711
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  267 non-null    int64  
 1   Pclass    267 non-null    int64  
 2   Age       267 non-null    float64
 3   SibSp     267 non-null    int64  
 4   Parch     267 non-null    int64  
 5   Fare      267 non-null    float64
 6   Sex       267 non-null    float64
 7   Emb_Q     267 non-null    float64
 8   Emb_S     267 non-null    float64
dtypes: float64(5), int64(4)
memory usage: 20.9 KB


In [10]:
train_features = train.drop('Survived', axis=1)
train_target = train['Survived']
test_features = test.drop('Survived', axis=1)
test_target = test['Survived']

Преобразуем некатегориальные данные отдельно в тренировочном и тестовом фрагментах..

In [11]:
scaler = StandardScaler()

In [12]:
colum_scal = set(set(train_features)) - set(['Survived','Sex','Emb_Q','Emb_S'])
colum_scal =list(colum_scal)
colum_scal

['Age', 'Parch', 'SibSp', 'Pclass', 'Fare']

In [13]:
train_features[colum_scal] = scaler.fit_transform(train_features[colum_scal])
df_train_features=pd.DataFrame(train_features)
df_train_features.head()

,Pclass,Age,SibSp,Parch,Fare,Sex,Emb_Q,Emb_S
638,0.819005,-0.086432,0.469973,-0.475742,-0.322469,1.0,0.0,1.0
473,0.819005,-0.545867,-0.473005,-0.475742,-0.447627,0.0,0.0,1.0
168,0.819005,-0.086432,-0.473005,-0.475742,0.484848,1.0,0.0,1.0
121,-0.368457,0.258144,0.469973,-0.475742,-0.043260,1.0,0.0,0.0
318,-1.555919,0.832437,0.469973,0.839421,2.043777,0.0,0.0,0.0


In [14]:
test_features[colum_scal] = scaler.fit_transform(test_features[colum_scal])
df_test_features=pd.DataFrame(test_features)
df_test_features.head()

,Pclass,Age,SibSp,Parch,Fare,Sex,Emb_Q,Emb_S
150,-1.611992,-0.606640,0.354209,-0.477248,0.713641,0.0,0.0,1.0
266,0.840440,-0.371792,0.354209,-0.477248,-0.491761,1.0,0.0,1.0
415,-0.385776,0.332751,0.354209,0.630798,0.014887,0.0,0.0,1.0
141,0.840440,-0.450075,0.354209,-0.477248,-0.326293,0.0,0.0,1.0
648,0.840440,-0.528357,-0.482728,-0.477248,-0.496371,0.0,0.0,1.0


Обучим модели.
---





---


Классификатор логистической регрессии с параметрами по умолчанию



In [15]:
lregr=LogisticRegression()

In [16]:
lregr.fit(train_features, train_target.values)

LogisticRegression()

In [17]:
print("Test error: %.6f" % (accuracy_score(train_target.values, lregr.predict(train_features))))

Test error: 0.797428


Полученное значение 0.797... означает, что 79,7% примеров в тренировочном фрагменте классифицированы правильно

In [18]:
print(classification_report(test_target.values, lregr.predict(test_features)))

              precision    recall  f1-score   support

           0       0.86      0.80      0.82       171
           1       0.68      0.76      0.72        96

    accuracy                           0.78       267
   macro avg       0.77      0.78      0.77       267
weighted avg       0.79      0.78      0.79       267



Полученные значения означают:

precision для "0" - 0.86 - из всех людей, для которых модель предсказала гибель на самом деле погибло 86%;

precision для "1" - 0.68 - из всех людей для которых модель предсказала что они выживут выжило 68%;

recall для "0" - 0.80 - из всех погибших людей (171) модель угадала, что они погибнут для 80%;

recall для "1" - 0.76 - из всех выживших людей (96) модель угадала, что они выживут для 76%;

f1-score - это средневзвешенное гармоническое значение precision и recall. Чем ближе оно к 1, тем лучше модель предсказывает результат (кто выживет, а кто погибнет). В нашем случае 72% для выживших и 82% для погибших;

support - количество погибших (171) и выжившых (96) на самом деле.

Итоговое значение accuracy f1-score - 0.78, что достаточно близко к тестовому значению 0.79

Разница в значениях для "0" и "1" по precision и f1-score обусловлена скорее всего несбалансированным датасетом и незначительным объемом данных.




---


Классификатор, реализующий голосование k ближайших соседей с параметрами по умолчанию



In [19]:
knn = KNeighborsClassifier()

In [20]:
knn.fit(train_features, train_target.values)

KNeighborsClassifier()

In [21]:
print("Test error: %.7f" % (accuracy_score(train_target.values, knn.predict(train_features))))

Test error: 0.8488746


Полученное значение 0.8488 означает, что 84,9% примеров в тренировочном фрагменте классифицированы правильно

In [22]:
print(classification_report(test_target.values, knn.predict(test_features)))

              precision    recall  f1-score   support

           0       0.83      0.77      0.80       171
           1       0.64      0.72      0.68        96

    accuracy                           0.75       267
   macro avg       0.73      0.75      0.74       267
weighted avg       0.76      0.75      0.76       267



Получены значения означают:

precision для "0" - 0.83 - из всех людей, для которых модель     предсказала гибель на самом деле погибло 83%;

precision для "1" - 0.64 - из всех людей для которых модель     предсказала что они выживут выжило 64%;

recall для "0" - 0.77 - из всех погибших людей (171) модель угадала, что они погибнут для 77%;

recall для "1" - 0.72 - из всех выживших людей (96) модель угадала, что они выживут для 72%;

f1-score - это средневзвешенное гармоническое значение precision и recall. Чем ближе оно к 1, тем лучше модель предсказывает результат (кто выживет, а кто погибнет). В нашем случае 68% для выживших и 80% для погибших;

support - количество погибших (171) и выжившых (96) на самом деле.

Итоговое значение accuracy f1-score - 0.75, меньше тестового значения 0.84. Это, а также разница в значениях для "0" и "1" по precision и f1-score обусловлена скорее всего несбалансированным датасетом и незначительным объемом данных.


Подберем гиперпараметры для обеих моделей, используя RandomizedSearchCV.
---



LogisticRegression

In [23]:
c_array = np.logspace(0, 1.2, 25)


In [25]:
parametr_lr={'penalty': ['l1', 'l2', 'None', 'elasticnet'],
                'class_weight': ['balanced', 'None'],
                'C': c_array,
                'solver': ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'],
                'max_iter': [120]}

In [26]:
lr_rs = RandomizedSearchCV(estimator=lregr, param_distributions=parametr_lr, cv=10, n_iter=800, random_state=42)

In [27]:
%%time
lr_rs.fit(train_features, train_target.values)

CPU times: user 29.6 s, sys: 250 ms, total: 29.9 s
Wall time: 29.9 s


/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
5990 fits failed out of a total of 8000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
340 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py", line 447, in _check_solver
    raise ValueError(
ValueError: Sol

RandomizedSearchCV(cv=10, estimator=LogisticRegression(), n_iter=800,
                   param_distributions={'C': array([ 1.        ,  1.12201845,  1.25892541,  1.41253754,  1.58489319,
        1.77827941,  1.99526231,  2.23872114,  2.51188643,  2.81838293,
        3.16227766,  3.54813389,  3.98107171,  4.46683592,  5.01187234,
        5.62341325,  6.30957344,  7.07945784,  7.94328235,  8.91250938,
       10.        , 11.22018454, 12.58925412, 14.12537545, 15.84893192]),
                                        'class_weight': ['balanced', 'None'],
                                        'max_iter': [120],
                                        'penalty': ['l1', 'l2', 'None',
                                                    'elasticnet'],
                                        'solver': ['lbfgs', 'liblinear',
                                                   'newton-cg',
                                                   'newton-cholesky', 'sag',
                                 

Так как не все параметры могут стыковаться друг с другом, при некоторых сочетаниях тест не дал результатов.




Выведем лучшее сочетание параметров.

In [28]:
best_params_lr = lr_rs.best_params_
best_params_lr

{'solver': 'liblinear',
 'penalty': 'l2',
 'max_iter': 120,
 'class_weight': 'balanced',
 'C': 1.7782794100389228}

Обучим модель на лучшем сочетании параметров.

In [29]:
lr_best=LogisticRegression(penalty='l2', solver='liblinear', class_weight='balanced', C=1.7782794100389228, max_iter=120)

In [30]:
%%time
lr_best.fit(train_features, train_target.values)

CPU times: user 5.88 ms, sys: 0 ns, total: 5.88 ms
Wall time: 7.01 ms


LogisticRegression(C=1.7782794100389228, class_weight='balanced', max_iter=120,
                   solver='liblinear')

In [31]:
print("Test error: %.7f" % (accuracy_score(train_target.values, lr_best.predict(train_features))))

Test error: 0.8006431


Полученное значение 0.8006... означает, что 80,1% примеров в тренировочном фрагменте классифицированы правильно. Результат практически такой-же, как при обучении с параметрами по умолчанию - 79,7%

In [32]:
print(classification_report(test_target.values, lr_best.predict(test_features)))

              precision    recall  f1-score   support

           0       0.86      0.75      0.80       171
           1       0.64      0.79      0.71        96

    accuracy                           0.76       267
   macro avg       0.75      0.77      0.75       267
weighted avg       0.78      0.76      0.77       267



Полученные метрики оказались несколько хуже, чем на моделях с параметрами по умолчанию. Скорее всего это обусловлено тем, что в те 800 вариантов, которые посчитал RandomizedSearchCV не вошли самые оптимальные соотношения. Но всё равно эти значения достаточно близки.

KNeighborsClassifier

In [33]:
parametr_knn = {'n_neighbors': range(1, 8),
                'weights': ['uniform', 'distance'],
                'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                'leaf_size': range(2, 40, 2),
                'p': [1, 2]}

In [34]:
knn_rs = RandomizedSearchCV(estimator=knn, param_distributions=parametr_knn, cv=5, n_iter=500, n_jobs=-1)

In [35]:
%%time
knn_rs.fit(train_features, train_target.values)

CPU times: user 1.28 s, sys: 108 ms, total: 1.39 s
Wall time: 17.4 s


RandomizedSearchCV(cv=5, estimator=KNeighborsClassifier(), n_iter=500,
                   n_jobs=-1,
                   param_distributions={'algorithm': ['auto', 'ball_tree',
                                                      'kd_tree', 'brute'],
                                        'leaf_size': range(2, 40, 2),
                                        'n_neighbors': range(1, 8), 'p': [1, 2],
                                        'weights': ['uniform', 'distance']})

Выведем лучшее сочетание параметров.

In [36]:
best_params = knn_rs.best_params_
best_params

{'weights': 'uniform',
 'p': 1,
 'n_neighbors': 7,
 'leaf_size': 22,
 'algorithm': 'auto'}

In [37]:
knn_best = KNeighborsClassifier(weights='uniform', algorithm='auto', p=1, n_neighbors=7, leaf_size=22)

In [38]:
knn_best.fit(train_features, train_target.values)

KNeighborsClassifier(leaf_size=22, n_neighbors=7, p=1)

In [39]:
print("Test error: %.7f" % (accuracy_score(train_target.values, knn_best.predict(train_features))))

Test error: 0.8376206


Полученное значение 0.8376... означает, что 83,8% примеров в тренировочном фрагменте классифицированы правильно. Результат практически такой-же, как при обучении с параметрами по умолчанию - 84,9%

In [40]:
print(classification_report(test_target.values, knn_best.predict(test_features)))

              precision    recall  f1-score   support

           0       0.84      0.85      0.85       171
           1       0.73      0.72      0.72        96

    accuracy                           0.80       267
   macro avg       0.78      0.78      0.78       267
weighted avg       0.80      0.80      0.80       267



Полученные метрики оказались лучше, чем на модели с параметрами по умолчанию. 

---
ИТОГ: в ходе выполнения задания было определено, что лучшим классификатором для анализа предложенных данных является KNeighborsClassifier с подобранными параметрами, т.к. метрики на тестовом фрагменте у данного классификатора лучше, чем у Logistic Regression.
---



